<a href="https://colab.research.google.com/github/ayoubouafdi/BigDataLabs/blob/main/TP_Spark_Streaming.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyspark

In [19]:
import os
# Création du dossier qui va recevoir les fichiers texte
os.makedirs("/content/mon_flux/", exist_ok=True)
import socket, time, threading

def start_socket_server():
    host = "localhost"
    port = 9999
    s = socket.socket()
    s.bind((host, port))
    s.listen(1)
    conn, addr = s.accept()
    messages = ["spark streaming dstream", "spark spark streaming", "big data spark"]
    while True:
        for msg in messages:
            conn.send((msg + "\n").encode())
            time.sleep(2)

# Lancement du serveur dans un thread séparé
threading.Thread(target=start_socket_server, daemon=True).start()

Exception in thread Thread-17 (start_socket_server):
Traceback (most recent call last):
  File "/usr/lib/python3.12/threading.py", line 1075, in _bootstrap_inner
    self.run()
  File "/usr/lib/python3.12/threading.py", line 1012, in run
    self._target(*self._args, **self._kwargs)
  File "/tmp/ipython-input-3035698602.py", line 10, in start_socket_server
OSError: [Errno 98] Address already in use


In [20]:
from pyspark import SparkContext
from pyspark.streaming import StreamingContext

sc = SparkContext.getOrCreate()
ssc = StreamingContext(sc, 5) # 5 secondes est la durée du batch

In [21]:
#Lire depuis le socket local sur le port 9999.
lines = ssc.textFileStream("/content/mon_flux/")

#Détailler les lignes en mots (flatMap).
words = lines.flatMap(lambda line: line.split(" "))

#Transformer chaque mot en paire (mot, 1)
pairs = words.map(lambda w: (w, 1))

#Réduire par clé pour additionner les occurrences.
counts = pairs.reduceByKey(lambda a, b: a + b)

#Affichage des résultats
counts.pprint()

In [22]:
ssc.start()
ssc.awaitTerminationOrTimeout(30)
ssc.stop(stopSparkContext=False)

-------------------------------------------
Time: 2026-01-27 20:16:25
-------------------------------------------

-------------------------------------------
Time: 2026-01-27 20:16:30
-------------------------------------------

-------------------------------------------
Time: 2026-01-27 20:16:35
-------------------------------------------

-------------------------------------------
Time: 2026-01-27 20:16:40
-------------------------------------------
('AYOUB', 1)
('OUAFDI', 1)
('IDS', 1)

-------------------------------------------
Time: 2026-01-27 20:16:45
-------------------------------------------

-------------------------------------------
Time: 2026-01-27 20:16:50
-------------------------------------------



In [23]:

# RÉPONSES AUX QUESTIONS DU TP - PAR AYOUB


# 1 et 2. Organisation du TP
# Le TP est divisé en sections : installation, serveur socket,
# création du contexte Spark, traitement WordCount et exécution.
# Chaque partie du code est commentée pour expliquer l'étape.

# 3. Modification pour la lecture depuis un fichier
# Pour lire des fichiers au lieu d'un socket, j'ai remplacé :
# lines = ssc.socketTextStream("localhost", 9999)
# par :
# lines = ssc.textFileStream("/content/mon_flux/")
# Cela permet à Spark de scanner un dossier et de traiter les nouveaux fichiers.

# 4. Qu'est-ce qu'un micro-batch ?
# C'est la façon dont Spark Streaming travaille : il ne traite pas les
# données une par une, mais il attend quelques secondes pour regrouper
# les données reçues dans un petit lot (batch) avant de les calculer.

# 5. Structure de base d'un DStream
# Un DStream est en fait une suite continue de RDD (Resilient Distributed Datasets).
# Chaque RDD représente les données arrivées pendant l'intervalle du batch.

# 6. Durée du batch
# Dans ce TP, on a utilisé une durée de 5 secondes. C'est le temps que Spark
# attend avant de traiter les mots reçus et d'afficher le résultat.
